Unzip dataset

In [ ]:
import zipfile
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

Import libraries

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import matplotlib.pyplot as plt


Read dataset (images and labels)

In [ ]:
cnt_train = 6000
cnt_test = 10

label_id = {'cat' : 0, 'dog' : 1}
data_path = '/kaggle/working/train'
def read_data(data_type):
    images = []
    labels = []
    cnt = {0 : 0, 1 : 0}
    for idx,filename in enumerate(os.listdir(data_path)):
        if(data_type == 0 and idx == cnt_train):
            break
        if(data_type == 1 and idx == cnt_train + cnt_test):
            break
        if(data_type == 1 and idx < cnt_train):
            continue  
        path = data_path + "/" + filename
        img = cv2.imread(path)
        cate = filename.split(".")[0]
        images.append(img)
        labels.append(label_id[cate])
        cnt[label_id[cate]]+=1
    print(cnt[0],cnt[1])
    return images,labels

train_images,y_train = read_data(0)  
test_images,y_test = read_data(1) 

In [ ]:
print(len(train_images),len(y_train))
print(len(test_images),len(y_test))

Extract features and descriptors from the images with SIFT

In [ ]:
def extract_sift_features(images):
    all_descriptors = []
    descriptors = []
    sift = cv2.xfeatures2d.SIFT_create()
    for img in tqdm(images):
        kp, des = sift.detectAndCompute(img,None)
        all_descriptors.extend(des)
        descriptors.append(des)
    return all_descriptors,descriptors

all_descriptors,train_descriptors = extract_sift_features(train_images) 
_,test_descriptors = extract_sift_features(test_images)

Train KMeans clustering model on all the descriptors list

In [ ]:
kmeans = KMeans(n_clusters = 75, max_iter=10,verbose=1,n_init=2)
kmeans.fit(all_descriptors)
cluster_centers = kmeans.cluster_centers_ 
print(cluster_centers)

In [ ]:
print(cluster_centers.shape)

Convert the images into histograms according to the clusters

In [ ]:
def convert_features_to_vbow(descriptors):
    vbow=[]
    for descriptor in tqdm(descriptors):
        histogram = np.zeros(len(cluster_centers))
        indexes = kmeans.predict(descriptor.astype('float'))
        for idx in indexes:
            histogram[idx]+=1
        vbow.append(histogram)
    return vbow

x_train = convert_features_to_vbow(train_descriptors) 
x_test = convert_features_to_vbow(test_descriptors)


Change datatype into numpy array

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(y_train.shape)
print(x_train.shape)

In [ ]:
print(x_train[0],y_train[0])

In [ ]:
res = []
query_idx = 7
for idx,img in enumerate(x_train):
    cos_sim = cosine_similarity(img.reshape(1,-1),x_test[query_idx].reshape(1,-1))
    res.append([cos_sim[0][0],idx])
resdf = pd.DataFrame(res,columns = ['similarity','index'])
resdf = resdf.sort_values('similarity', ascending = False)

plt.figure(figsize=(20,10))

plt.subplot(3,5,3)
plt.imshow(cv2.cvtColor(test_images[query_idx],cv2.COLOR_BGR2RGB))
plt.title('Query')
plt.xticks([])
plt.yticks([])
for index,[idx,res] in enumerate(resdf.iterrows()):
    if(index>=10):
        break
    print(res['similarity'],res['index'])
    plt.subplot(3,5,index+6)
    plt.imshow(cv2.cvtColor(train_images[res['index'].astype('int')],cv2.COLOR_BGR2RGB))
    plt.title('Similarity: ' + str(res['similarity']))
    plt.xticks([])
    plt.yticks([])